In [1]:
!pip install gradio transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr
import re

model_path = "prajjwal888/Llama-2-7b-chat-finetune"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
# Parse model output
def parse_generated_text(text: str) -> dict:
    clean_text = re.sub(r"\[/?INST\]", "", text)
    clean_text = re.sub(r"Question:\s*Question:", "Question:", clean_text)
    clean_text = clean_text.strip()

    match = re.search(r"Question:\s*(.*?)(?:\nHint:|Hint:)(.*)", clean_text, re.DOTALL)

    if match:
        question = match.group(1).strip().strip('"').replace("Question:", "").strip()
        hint = match.group(2).strip().strip('"')
    else:
        question = clean_text.strip()
        hint = "No hint available"

    return {
        "question": question,
        "hint": hint
    }

# Generation function
def generate_questions(topic, difficulty, types, count):
    print("Received input:", topic, difficulty, types, count)  # ✅ Debug print

    try:
        pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            # device=0,
            max_length=200,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

        questions = []

        for _ in range(count):
            for q_type in types:
                prompt = (
                    f"Generate a {difficulty} difficulty {q_type} question about {topic}.\n"
                    "Format strictly as follows:\n"
                    "Question: <your question here>\n"
                    "Hint: <your hint here or 'No hint available'>"
                )

                formatted_prompt = f"<s>[INST] {prompt} [/INST]"
                print("Prompt:", formatted_prompt)

                result = pipe(formatted_prompt)
                print("Raw Output:", result)

                generated_text = result[0]['generated_text'].replace(formatted_prompt, "").strip()
                parsed = parse_generated_text(generated_text)

                questions.append(f"**Type**: {q_type}\n\n**Question**: {parsed['question']}\n\n**Hint**: {parsed['hint']}\n\n---")

        return "\n\n".join(questions)

    except Exception as e:
        print("Error:", e)
        return f"Something went wrong: {e}"

# Gradio UI
iface = gr.Interface(
    fn=generate_questions,
    inputs=[
        gr.Textbox(label="Topic"),
        gr.Dropdown(choices=["easy", "medium", "hard"], label="Difficulty", value="medium"),
        gr.CheckboxGroup(choices=["Conceptual", "Numerical", "Application"], label="Question Types"),
        gr.Slider(minimum=1, maximum=5, step=1, value=2, label="Number of Questions per Type")
    ],
    outputs=gr.Markdown(label="Generated Questions"),
    title="AI Question Generator",
    description="Enter a topic, select difficulty and question types to generate AI-powered questions."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://186a69a574f95f1570.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
